In [1]:
import os
import pandas as pd
import io
from ipywidgets import FileUpload, Dropdown, Output, VBox, Button
from IPython.display import display
import pygwalker as pyg

DATA_DIR = "./datas"  # Directory to store uploaded CSV
os.makedirs(DATA_DIR, exist_ok=True)

# Create widgets
uploader = FileUpload(accept='.csv', multiple=False)
dropdown = Dropdown(options=["Please select data"], description="Select data:")
out = Output()
visualize_btn = Button(description="Generate visualization")

# Handle file upload
def handle_upload(change):
    if uploader.value:
        # Get uploaded file content
        uploaded_file = uploader.value[0] if isinstance(uploader.value, tuple) else next(iter(uploader.value.values()))
        content_bytes = uploaded_file['content'].tobytes()

        # Read CSV data
        df = pd.read_csv(io.StringIO(content_bytes.decode('utf-8')))

        # Get file name and save
        file_name = uploaded_file.get('name', 'uploaded_file.csv')
        file_path = os.path.join(DATA_DIR, file_name)
        df.to_csv(file_path, index=False)

        print(f"File saved as: {file_path}")
        
        # Update dropdown list
        update_dropdown()
    else:
        print("⚠️ Please upload a CSV file!")

# Update dropdown with file list
def update_dropdown():
    csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
    if csv_files:
        dropdown.options = csv_files
    else:
        dropdown.options = ["Please select data"]

# Visualize CSV when selected
def visualize(change):
    with out:
        out.clear_output()
        if dropdown.value != "Please select data":
            file_path = os.path.join(DATA_DIR, dropdown.value)
            df = pd.read_csv(file_path)
            display(pyg.walk(df, env='Jupyter'))
        else:
            print("⚠️ Please select a CSV file!")

# Observe events and bind actions
uploader.observe(handle_upload, names='value')
visualize_btn.on_click(visualize)

# Display the widgets
display(VBox([uploader, dropdown, visualize_btn, out]))

# Initialize the dropdown list
update_dropdown()
